In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
dataset= pd.read_csv("C:/Users/Dario/Desktop/dario/DATA SCIENCE/Algorithmic Methods of Data Mining and laboratory/HOMEWORK-4/amazon_food_reviews/Reviews.csv")
dataset

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


# data cleaning

In [3]:
dataset.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [4]:
dataset["ProfileName"].fillna("",inplace=True)
dataset["Summary"].fillna("",inplace=True)

In [5]:
dataset.set_index("Id",inplace=True)
#Id diventa il nostro indice

In [6]:
dataset['Datetime'] = dataset.Time.apply(lambda value: pd.to_datetime(value, unit='s'))
#create a new column with the Unix time stamp converted

## looking into the text

In [7]:
lemmatizer= WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [8]:
def clean_text(text):
    words = word_tokenize(text)    
    filtered_words = []
    for word in words:        
        if word.lower() not in stop_words and word.isalpha():
            filtered_words.append(lemmatizer.lemmatize(word.lower()))
    return filtered_words

In [9]:
wordlist= dataset["Summary"].apply(lambda text: clean_text(text))

In [48]:
dataset["subject_words"]=wordlist

In [49]:
vocabulary= [item for sublist in wordlist for item in sublist]

### total words

In [50]:
len(vocabulary)
#total words

1596148

TF-IDF

In [55]:
#initialize the column
dataset["subject_words_str"]=1

In [56]:
for i in range(len(dataset)):
    dataset["subject_words_str"].iloc[i]= " ".join(dataset["subject_words"].iloc[i])

C:\Users\Dario\anaconda3\envs\DataScience\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [57]:
dataset["subject_words_str"]

Id
1                  good quality dog food
2                             advertised
3                            delight say
4                         cough medicine
5                            great taffy
                       ...              
568450                           without
568451                      disappointed
568452                  perfect maltipoo
568453    favorite training reward treat
568454                       great honey
Name: subject_words_str, Length: 568454, dtype: object

In [58]:
vectorizer = TfidfVectorizer()

In [59]:
X = vectorizer.fit_transform(dataset["subject_words_str"]) #You can interpret this as "(sentence_index, feature_index) count"
# it's a scipy.sparse.csr.csr_matrix object

In [60]:
print(X)


  (0, 9733)	0.5151467043771162
  (0, 7402)	0.46955967149711125
  (0, 20381)	0.6123002580709537
  (0, 10811)	0.37312997981824597
  (1, 326)	1.0
  (2, 22168)	0.6659348750138963
  (2, 6651)	0.7460098807926249
  (3, 15777)	0.6957391412044929
  (3, 5674)	0.7182945408368595
  (4, 25141)	0.9487159635753054
  (4, 11078)	0.31612975256590414
  (5, 17150)	0.5131783556254328
  (5, 25141)	0.8582819905588004
  (6, 2950)	0.6573609260488061
  (6, 8773)	0.6143113372659096
  (6, 11078)	0.292596947048592
  (6, 10811)	0.3238598777074267
  (7, 25346)	0.41395089079318736
  (7, 28326)	0.47144694735565085
  (7, 25141)	0.7787056156472514
  (8, 1856)	0.7288711105776888
  (8, 28615)	0.6846509359996863
  (9, 11734)	0.6451343831739267
  (9, 9733)	0.5646858764118357
  (9, 7402)	0.5147149586205885
  :	:
  (568444, 2254)	0.2844063625364668
  (568445, 9433)	0.6361420289875368
  (568445, 19679)	0.5599151519434271
  (568445, 23902)	0.5308656530420751
  (568446, 16284)	0.7474302588793619
  (568446, 28501)	0.6643402803620

In [61]:
print(vectorizer.get_feature_names())

['aa', 'aaa', 'aaaaaaaaaaacccckkkkk', 'aaaaaaaaaaahhhhhhhhhhhhh', 'aaaaaaaah', 'aaaaaaggggggghhhhhhhh', 'aaaaaagggghhhh', 'aaaaaagh', 'aaaaaamazing', 'aaaaahhh', 'aaaah', 'aaaahhh', 'aaaahhhh', 'aaaahhhhh', 'aaaamaaaazing', 'aaaamazing', 'aaaammmmaaaaazziiiinnnnggg', 'aaaarrrrrgggghhhhh', 'aaaccchhh', 'aaah', 'aaahhh', 'aaahhhh', 'aaahhhhhh', 'aaamazing', 'aachener', 'aack', 'aacute', 'aah', 'aahey', 'aahh', 'aamzing', 'aardvark', 'aat', 'aaverage', 'aawesome', 'ab', 'aback', 'abacore', 'abalone', 'abaloo', 'abandon', 'abb', 'abba', 'abbey', 'abbie', 'abbott', 'abby', 'abc', 'abd', 'abdominal', 'abel', 'abfab', 'abhor', 'abi', 'abide', 'ability', 'abit', 'abj', 'abject', 'able', 'aboard', 'abolutely', 'abomination', 'aborio', 'aboslutely', 'abosolutely', 'abot', 'abound', 'abounds', 'abra', 'abroad', 'absence', 'absent', 'absente', 'absinthe', 'abslutely', 'abso', 'absoloutely', 'absoloutly', 'absoluetly', 'absolute', 'absolutel', 'absolutely', 'absolutey', 'absolutlely', 'absolutley',

### unique words

In [62]:
len(vectorizer.get_feature_names())

29082

tf-idf first document

In [63]:
first_vector_tfidfvectorizer=X[0]          

In [64]:
df_1 = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=vectorizer.get_feature_names(), columns=["tfidf"]) 
df_1 =df_1.sort_values(by=["tfidf"],ascending=False)
df_1


,tfidf
quality,0.612300
food,0.515147
dog,0.469560
good,0.373130
pmetta,0.000000
...,...
fofanalana,0.000000
foe,0.000000
fodder,0.000000
fod,0.000000
